In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Experiment')

In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from googletrans import Translator
import pandas as pd
import time

In [ ]:
def preprocessing_dataset(dataset):
    """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []
    for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
        i = i[1:-1].split(',')[0].split(':')[1]
        j = j[1:-1].split(',')[0].split(':')[1]

        subject_entity.append(i)
        object_entity.append(j)
    out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
    return out_dataset


def load_data(dataset_dir):
    """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)
    
    return dataset

In [ ]:
df = load_data("./dataset/train/train.csv")
df = df.sample(frac=0.3).reset_index()

In [ ]:
subject_entity = []
object_entity = []
sentences = []
ids = []
labels = []
id = len(df['id'])

locale_str_en = 'en'
locale_str_ko = 'ko'

start = time.monotonic()
for index, row in df.iterrows():
    translator = Translator()
    sent, sub, obj = row['sentence'], row['subject_entity'], row['object_entity']
    sub_ = sub.replace("'", "")
    obj_ = obj.replace("'", "")
    en_result = translator.translate(sent, dest=locale_str_en)
    ko_result = translator.translate(en_result.text, dest=locale_str_ko)
    ko_result = ko_result.text

    if sub_ in ko_result and obj_ in ko_result:
        if ko_result == sent:
            print(f'{id} :: original={sent} | generated={ko_result}')
            continue
        sentences.append(ko_result)
        subject_entity.append(sub)
        object_entity.append(obj)
        ids.append(id)
        labels.append(row['label'])
        id += 1
end = time.monotonic()

ReadTimeout: ignored

In [ ]:
print('Elapsed: {}'.format(end-start))

new_df = pd.DataFrame({'id':ids, 'sentence':sentences,'subject_entity':subject_entity,'object_entity':object_entity,'label':labels,})
new_df.to_csv('augmented.csv')

In [ ]:
display(new_df.head())
new_df.describe